In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
from typing import Callable

## Experimentos

In [ ]:
sns.set_style("whitegrid")
df_resultados = pd.read_csv("resultados.csv")

directorio = 'img' 
try:
    os.mkdir(directorio) 
except OSError as error:
    pass
    #print(error)

In [ ]:
FIGSIZE = (10,5)

def comp_plt(
        df: pd.DataFrame,
        alg: str,
        func: Callable[[], float],
        func_label: str,
        dataset: str=None,
    ):
    
    ## Obtenemos la constante multiplicativa usando CML
    
    b = np.array(df['tiempo']).reshape(-1, 1)
    A = np.array([func(n) for n in df['n']]).reshape(-1, 1)
    AtA = A.T @ A
    Atb = A.T @ b

    c = np.linalg.solve(AtA, Atb)

    df['complejidad'] = func(df['n']) * c[0,0]

    
    plt.subplots(figsize=FIGSIZE)
    
    fig = sns.scatterplot(data=df, x='n', y='tiempo', hue='dataset')
    
    fig.set(xlabel='n', ylabel='tiempo (ms)');
    if dataset is None:
        fig.set_title(f'Tiempos de ejecución de {alg}')
        plt.savefig(f"img/{alg}_tiempos.svg")
    else:
        fig.set_title(f'Tiempos de ejecución de {alg} en {dataset}')
        plt.savefig(f"img/{alg}_{dataset}_tiempos.svg")

    
    fig = sns.lineplot(data=df, x='n', y='complejidad', color='orange')
    fig.set(xlabel='n', ylabel='tiempo (ms)');
    fig.legend(labels=[f'O({func_label})',alg]);
    if dataset is None:
        fig.set_title(f'Complejidad de {alg}')
        plt.savefig(f"img/{alg}.svg")
    else:
        fig.set_title(f'Complejidad de {alg} en {dataset}')
        plt.savefig(f"img/{alg}_{dataset}.svg")

    

def corr_plt(
        df: pd.DataFrame,
        alg:str,
        dataset: str=None,
    ):
    
    plt.figure(figsize=FIGSIZE)
    
    sns.lmplot(data=df, x='tiempo', y='complejidad', height=FIGSIZE[1], aspect=2);
    fig = plt.gca()
    fig.set(xlabel='tiempo (ms)', ylabel='complejidad');
    if dataset is None:
        fig.set_title(f'Correlación entre tiempos de ejecución y\n complejidad teórica de {alg}')
        plt.savefig(f"img/{alg}_corr.svg")
    else:
        fig.set_title(f'Correlación entre tiempos de ejecución y\n complejidad teórica de {alg} en {dataset}')
        plt.savefig(f"img/{alg}_{dataset}_corr.svg")


    print(np.corrcoef(df['tiempo'], df['complejidad'])[0,1])

## Funciones de complejidad

In [ ]:
def idt(x) -> float:
    return x

def exp(x) -> float:
    return 2**x

### Experimento 1: Complejidad fuerza bruta

En este experimento vamos a ver la complejidad del algoritmo de fuerza bruta con el dataset _fuerza-bruta_. La hipotesis es que esta va a coincidir con $\mathcal{O}(2^{n})$.

In [ ]:
df = df_resultados[df_resultados["metodo"] == "BF"].copy()

comp_plt(df, "FB", exp, "2^n")
corr_plt(df, "FB")

## Experimento 2: Complejidad backtracking 

In [ ]:
df = df_resultados[
    (df_resultados['metodo'] == 'BT') &
    (df_resultados['n'] <= 30) &
    ((df_resultados['dataset'] == 'peor-caso-bt') |
     (df_resultados['dataset'] == 'mejor-caso-bt'))].copy()

plt.subplots(figsize=FIGSIZE)
    
fig = sns.scatterplot(data=df, x='n', y='tiempo', hue='dataset')

fig.set(xlabel='n', ylabel='tiempo (ms)');
fig.set_title(f'Tiempos de ejecución de BT')

plt.savefig(f"img/BT_tiempos_dos_datasets.svg")

### Mejor caso (BT)

In [ ]:
df = df_resultados[
    (df_resultados['metodo'] == 'BT') &
    (df_resultados['n'] <= 30) &
    (df_resultados['dataset'] == 'mejor-caso-bt')].copy()

comp_plt(df, 'BT', idt, "n", "mejor-caso-bt")
corr_plt(df, 'BT', "mejor-caso-bt")

### Peor caso (BT)

In [ ]:
df = df_resultados[
    (df_resultados['metodo'] == 'BT') &
    (df_resultados['n'] <= 30) &
    (df_resultados['dataset'] == 'peor-caso-bt')].copy()

comp_plt(df, 'BT', exp, "2^n", "peor-caso-bt")
corr_plt(df, 'BT', "peor-caso-bt")

## Experimento 4: Complejidad PD

In [ ]:
df = df_resultados[
    (df_resultados['metodo'] == 'DP') &
    (df_resultados['dataset'] == 'muchos-productos')].copy()
df['complejidad'] = df['n'] * df['R'] * 0.000019

plt.subplots(figsize=FIGSIZE)
fig = sns.scatterplot(data=df, x='n', y='tiempo')

fig.set(xlabel='n', ylabel='tiempo (ms)');
fig.set_title(f'Tiempos de ejecución de DP')

plt.savefig(f"img/PD_tiempos.svg")

fig = sns.lineplot(data=df, x='n', y='complejidad', color='orange')
fig.set(xlabel='n', ylabel='tiempo (ms)');
fig.legend(labels=[f'O(n * R)',"DP"]);
fig.set_title(f'Complejidad de DP')
plt.savefig(f"img/PD.svg")

In [ ]:
df = df_resultados[
    (df_resultados['metodo'] == 'DP') &
    (df_resultados['dataset'] == 'muchos-productos')].copy()
df['complejidad'] = df['n'] * df['R']
corr_plt(df, "DP")

### Experimento : Comparación Backtracking - Programación dinámica
En este experimento queremos ver si existen instancias donde el método de backtracking, cuya complejidad teórica en el peór caso es de $O(2^{n})$, se comporta más eficientemente que el algoritmo de programación dinámica, cuya complejidad teórica en el peór caso es de $O(n \cdot R)$. Para ello, veremos que sucede con los tiempos de ejecución de estos algoritmos en los dataset **alta-superposicion** y **sin-superposicion**.

In [ ]:
df = df_resultados[
    ((df_resultados['metodo'] == 'BT') | (df_resultados['metodo'] == 'DP')) &
    (df_resultados['dataset'] == 'alta-superposicion')
]

In [ ]:
plt.subplots(figsize=FIGSIZE)
fig = sns.scatterplot(data=df, x='n', y='tiempo', hue='metodo')
fig.set(xlabel='n', ylabel='tiempo (ms)');
fig.set_title(f'Tiempos de ejecución de DP y BT')
plt.savefig(f"img/PD_BT_alta_tiempos.svg")

### - Dataset sin-superposicion 

In [ ]:
df = df_resultados[
    ((df_resultados['metodo'] == 'BT') | (df_resultados['metodo'] == 'DP')) &
    (df_resultados['dataset'] == 'sin-superposicion')
]

In [ ]:
plt.subplots(figsize=FIGSIZE)
fig = sns.scatterplot(data=df, x='n', y='tiempo', hue='metodo')
fig.set(xlabel='n', ylabel='tiempo (ms)');
fig.set_title(f'Tiempos de ejecución de DP y BT')
plt.savefig(f"img/PD_BT_sin_tiempos.svg")